In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque

mp_face_mesh = mp.solutions.face_mesh
mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                             refine_landmarks=True, min_detection_confidence=0.5,
                             min_tracking_confidence=0.5)

# Helper: EMA
class EMA:
    def __init__(self, alpha=0.5): self.alpha, self.prev = alpha, None
    def update(self, x): 
        self.prev = x if self.prev is None else self.alpha*x+(1-self.alpha)*self.prev
        return self.prev

def norm_dist(a, b): return np.linalg.norm(a-b)

# MediaPipe indices for lips and eyes (refine_landmarks=True improves lips)
LIP_LEFT = 61; LIP_RIGHT = 291    # corners
EYE_LEFT = 33; EYE_RIGHT = 263    # outer canthi
# Inner/outer lip points for MAR-like metric (approximate FaceMesh mapping)
UPPER = [13, 312, 82]   # sample upper lip points
LOWER = [14, 17, 87]    # sample lower lip points

sw_ema = EMA(0.5); mar_ema = EMA(0.5)
enter_sw, exit_sw = 0.52, 0.48
enter_mar, exit_mar = 0.34, 0.30
smiling = False

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret: break
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = mesh.process(img_rgb)
    if res.multi_face_landmarks:
        h, w = frame.shape[:2]
        lms = [(int(p.x*w), int(p.y*h)) for p in res.multi_face_landmarks[0].landmark]
        lip_left = np.array(lms[LIP_LEFT], dtype=np.float32)
        lip_right = np.array(lms[LIP_RIGHT], dtype=np.float32)
        eye_left = np.array(lms[EYE_LEFT], dtype=np.float32)
        eye_right = np.array(lms[EYE_RIGHT], dtype=np.float32)

        inter_ocular = norm_dist(eye_left, eye_right)
        sw = norm_dist(lip_left, lip_right) / (inter_ocular + 1e-6)

        # MAR-like from sampled lip pairs
        upper_pts = [np.array(lms[i], dtype=np.float32) for i in UPPER]
        lower_pts = [np.array(lms[i], dtype=np.float32) for i in LOWER]
        vertical = np.mean([norm_dist(u, l) for u, l in zip(upper_pts, lower_pts)])
        horizontal = norm_dist(lip_left, lip_right)
        mar = vertical / (horizontal + 1e-6)

        sw_s = sw_ema.update(sw)
        mar_s = mar_ema.update(mar)

        if (not smiling and sw_s > enter_sw and mar_s > enter_mar):
            smiling = True
        elif (smiling and (sw_s < exit_sw or mar_s < exit_mar)):
            smiling = False

        # Draw UI
        color = (0, 255, 0) if smiling else (0, 0, 255)
        cv2.line(frame, tuple(lip_left.astype(int)), tuple(lip_right.astype(int)), color, 2)
        cv2.putText(frame, f"SW:{sw_s:.2f} MAR:{mar_s:.2f} SMILE:{int(smiling)}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow("Smile detector - MediaPipe", frame)
    if cv2.waitKey(1) & 0xFF == 27: break

cap.release(); cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [1]:
import cv2, dlib, numpy as np, time

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def shape_to_np(shape):
    return np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)], dtype=np.float32)

def dist(a,b): return np.linalg.norm(a-b)

def mar_68(pts):
    vertical = dist(pts[51], pts[59]) + dist(pts[52], pts[58]) + dist(pts[53], pts[57])
    horizontal = dist(pts[48], pts[54])
    return vertical / (horizontal + 1e-6)

def smile_width(pts):
    return dist(pts[48], pts[54]) / (dist(pts[36], pts[45]) + 1e-6)

def lip_corner_angle(pts):
    dx = pts[54][0] - pts[48][0]
    dy = pts[54][1] - pts[48][1]
    return np.degrees(np.arctan2(dy, dx))

# EMA helper
class EMA:
    def __init__(self, alpha=0.5): self.alpha, self.prev = alpha, None
    def update(self, x):
        self.prev = x if self.prev is None else self.alpha*x+(1-self.alpha)*self.prev
        return self.prev

sw_ema, mar_ema, ang_ema = EMA(0.4), EMA(0.4), EMA(0.4)

# --- Calibration ---
print("Калибровка: сядь спокойно без улыбки (3 секунды)...")
cap = cv2.VideoCapture(0)
sw_vals, mar_vals, ang_vals = [], [], []
start = time.time()
while time.time()-start < 3:
    ret, frame = cap.read()
    if not ret: continue
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 0)
    if faces:
        shape = predictor(gray, faces[0])
        pts = shape_to_np(shape)
        sw_vals.append(smile_width(pts))
        mar_vals.append(mar_68(pts))
        ang_vals.append(lip_corner_angle(pts))
    cv2.imshow("Calibration", frame)
    if cv2.waitKey(1) & 0xFF == 27: break

cv2.destroyWindow("Calibration")
sw_base, mar_base, ang_base = np.mean(sw_vals), np.mean(mar_vals), np.mean(ang_vals)
print(f"База: SW={sw_base:.3f}, MAR={mar_base:.3f}, ANG={ang_base:.2f}")

# --- Detection loop ---
smiling = False
while True:
    ret, frame = cap.read()
    if not ret: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 0)
    if faces:
        shape = predictor(gray, faces[0])
        pts = shape_to_np(shape)

        sw = sw_ema.update(smile_width(pts))
        mar = mar_ema.update(mar_68(pts))
        ang = ang_ema.update(lip_corner_angle(pts))

        # Decision: рост ширины и MAR относительно базы + угол ближе к горизонтали
        if (sw > sw_base*1.1 and mar > mar_base*1.05 and abs(ang-ang_base) < 10):
            smiling = True
        else:
            smiling = False

        color = (0,255,0) if smiling else (0,0,255)
        cv2.putText(frame, f"SW:{sw:.2f} MAR:{mar:.2f} ANG:{ang:.1f} SMILE:{int(smiling)}",
                    (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.polylines(frame, [pts[48:60].astype(int)], True, color, 2)
        cv2.polylines(frame, [pts[60:68].astype(int)], True, color, 1)

    cv2.imshow("Smile detector", frame)
    if cv2.waitKey(1) & 0xFF == 27: break

cap.release(); cv2.destroyAllWindows()


Калибровка: сядь спокойно без улыбки (3 секунды)...
База: SW=0.541, MAR=1.182, ANG=4.82


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# Pose для рук
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# FaceMesh для глаз
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=False,
                             max_num_faces=1,
                             refine_landmarks=True,
                             min_detection_confidence=0.5,
                             min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

clap_count = 0
clap_state = False

def euclidean(a, b):
    return np.linalg.norm(a - b)

def eye_aspect_ratio(landmarks, eye_indices, w, h):
    pts = np.array([[landmarks[i].x * w, landmarks[i].y * h] for i in eye_indices])
    return (euclidean(pts[1], pts[5]) + euclidean(pts[2], pts[4])) / (2.0 * euclidean(pts[0], pts[3]) + 1e-6)

# Индексы для глаз
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# --- Фиксированный порог EAR ---
ear_thresh = 0.183   # твой рассчитанный порог

ear_buffer = deque(maxlen=5)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res_pose = pose.process(img_rgb)
    res_face = face_mesh.process(img_rgb)
    h, w = frame.shape[:2]

    # --- Pose: руки и хлопки ---
    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark
        left_shoulder = np.array([lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                                  lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h])
        left_wrist = np.array([lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w,
                               lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h])
        right_shoulder = np.array([lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * w,
                                   lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * h])
        right_wrist = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * w,
                                lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * h])

        left_up = left_wrist[1] < left_shoulder[1]
        right_up = right_wrist[1] < right_shoulder[1]

        status = []
        if left_up: status.append("Left hand up")
        if right_up: status.append("Right hand up")
        if not status: status.append("....")

        # хлопки
        dist_wrists = euclidean(left_wrist, right_wrist)
        if dist_wrists < 80:
            if not clap_state:
                clap_count += 1
                clap_state = True
        else:
            clap_state = False

        cv2.putText(frame, ", ".join(status), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Claps: {clap_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # --- FaceMesh: глаза ---
    if res_face.multi_face_landmarks:
        landmarks = res_face.multi_face_landmarks[0].landmark
        left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, w, h)
        right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, w, h)
        ear = (left_ear + right_ear) / 2.0

        ear_buffer.append(ear)
        ear_avg = np.mean(ear_buffer)

        if ear_avg < ear_thresh:
            eye_status = "Eyes closed"
        else:
            eye_status = "Eyes open"

        cv2.putText(frame, eye_status, (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Detection", frame)
    key = cv2.waitKey(1) & 0xFF
    if key in [27, ord('q')]:
        print("Остановка работы по клавише.")
        break

cap.release()
cv2.destroyAllWindows()


Калибровка: держи глаза ОТКРЫТЫМИ (3 секунды)...
Теперь ЗАКРОЙ глаза (3 секунды)...
EAR open=0.273, closed=0.098, threshold=0.185
Остановка работы по клавише.


In [11]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# --- Pose для рук ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# --- FaceMesh для глаз ---
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=False,
                             max_num_faces=1,
                             refine_landmarks=True,
                             min_detection_confidence=0.5,
                             min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

clap_count = 0
clap_state = False

def euclidean(a, b):
    return np.linalg.norm(a - b)

def eye_aspect_ratio(landmarks, eye_indices, w, h):
    pts = np.array([[landmarks[i].x * w, landmarks[i].y * h] for i in eye_indices])
    return (euclidean(pts[1], pts[5]) + euclidean(pts[2], pts[4])) / (2.0 * euclidean(pts[0], pts[3]) + 1e-6)

# Индексы для глаз (FaceMesh)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# --- Фиксированный порог EAR (из твоей калибровки) ---
ear_thresh = 0.183

# Небольшое сглаживание EAR
ear_buffer = deque(maxlen=5)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res_pose = pose.process(img_rgb)
    res_face = face_mesh.process(img_rgb)
    h, w = frame.shape[:2]

    # --- Руки и хлопки ---
    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark

        left_shoulder = np.array([lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                                  lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h])
        left_wrist = np.array([lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w,
                               lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h])

        right_shoulder = np.array([lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * w,
                                   lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * h])
        right_wrist = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * w,
                                lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * h])

        left_up = left_wrist[1] < left_shoulder[1]
        right_up = right_wrist[1] < right_shoulder[1]

        status = []
        if left_up: status.append("Left hand up")
        if right_up: status.append("Right hand up")
        if not status: status.append("....")

        # Хлопки: расстояние между запястьями
        dist_wrists = euclidean(left_wrist, right_wrist)
        if dist_wrists < 80:
            if not clap_state:
                clap_count += 1
                clap_state = True
        else:
            clap_state = False

        cv2.putText(frame, ", ".join(status), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Claps: {clap_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # --- Глаза по FaceMesh и фиксированному порогу EAR ---
    if res_face.multi_face_landmarks:
        landmarks = res_face.multi_face_landmarks[0].landmark
        left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, w, h)
        right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, w, h)
        ear = (left_ear + right_ear) / 2.0

        ear_buffer.append(ear)
        ear_avg = np.mean(ear_buffer)

        eye_status = "Eyes closed" if ear_avg < ear_thresh else "Eyes open"
        cv2.putText(frame, eye_status, (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Detection", frame)
    key = cv2.waitKey(1) & 0xFF
    if key in [27, ord('q')]:
        print("Остановка работы по клавише.")
        break

cap.release()
cv2.destroyAllWindows()


Остановка работы по клавише.


In [12]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
import mediapipe as mp
from collections import deque

# --- Pose для рук ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# --- MTCNN для лица ---
detector = MTCNN()

cap = cv2.VideoCapture(0)

clap_count = 0
clap_state = False

# --- EAR порог (из твоей калибровки) ---
ear_thresh = 0.183
ear_buffer = deque(maxlen=5)

def euclidean(a, b):
    return np.linalg.norm(a - b)

def eye_aspect_ratio(left_eye, right_eye):
    # У MTCNN глаза — это одна точка (центр). EAR классический не посчитаем.
    # Поэтому используем расстояние между глазами как стабильный ориентир.
    return euclidean(left_eye, right_eye)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]

    # --- Pose: руки и хлопки ---
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res_pose = pose.process(img_rgb)

    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark
        left_shoulder = np.array([lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                                  lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h])
        left_wrist = np.array([lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w,
                               lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h])
        right_shoulder = np.array([lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * w,
                                   lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * h])
        right_wrist = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * w,
                                lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * h])

        left_up = left_wrist[1] < left_shoulder[1]
        right_up = right_wrist[1] < right_shoulder[1]

        status = []
        if left_up: status.append("Left hand up")
        if right_up: status.append("Right hand up")
        if not status: status.append("....")

        # хлопки
        dist_wrists = euclidean(left_wrist, right_wrist)
        if dist_wrists < 80:
            if not clap_state:
                clap_count += 1
                clap_state = True
        else:
            clap_state = False

        cv2.putText(frame, ", ".join(status), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Claps: {clap_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # --- MTCNN: глаза ---
    faces = detector.detect_faces(frame)
    if faces:
        face = faces[0]
        keypoints = face['keypoints']
        left_eye = np.array(keypoints['left_eye'])
        right_eye = np.array(keypoints['right_eye'])

        # EAR surrogate: расстояние по вертикали (глаза "закрыты" → меньше контраст)
        eye_dist = euclidean(left_eye, right_eye)
        ear_buffer.append(eye_dist)
        ear_avg = np.mean(ear_buffer)

        # Здесь вместо классического EAR используем порог на "eye_dist"
        # Для реальной работы лучше калибровать отдельно
        if ear_avg < 40:  # эмпирический порог, зависит от камеры
            eye_status = "Eyes closed"
        else:
            eye_status = "Eyes open"

        cv2.putText(frame, eye_status, (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Detection MTCNN+Pose", frame)
    key = cv2.waitKey(1) & 0xFF
    if key in [27, ord('q')]:
        print("Остановка работы по клавише.")
        break

cap.release()
cv2.destroyAllWindows()


d:\tms\new_venv\.venv\lib\site-packages\mtcnn\mtcnn.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources




1/1 [==============================] - 0s 18ms/step
Остановка работы по клавише.


In [14]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
import mediapipe as mp
from collections import deque

# --- Pose для рук ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# --- FaceMesh для глаз (на всём кадре) ---
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=False,
                             max_num_faces=1,
                             refine_landmarks=True,
                             min_detection_confidence=0.5,
                             min_tracking_confidence=0.5)

# --- MTCNN для лица (bbox и ключевые точки) ---
detector = MTCNN()

cap = cv2.VideoCapture(0)

# --- Порог EAR (из твоей калибровки) ---
ear_thresh = 0.183
ear_buffer = deque(maxlen=5)

# --- Хлопки: параметры гистерезиса и скорости ---
clap_count = 0
clap_state = False           # чтобы не считать один хлопок много раз
prev_dist_norm = None        # нормализованная дистанция в предыдущем кадре
approach_buffer = deque(maxlen=4)  # скорость сближения рук (для устойчивости)
enter_thresh = 0.12          # порог входа (руки близко), в долях ширины кадра
exit_thresh = 0.16           # порог выхода (руки разошлись), в долях ширины кадра
min_approach = 0.02          # минимальная скорость сближения (нормализованная) для фиксации хлопка

def euclidean(a, b):
    return np.linalg.norm(a - b)

def eye_aspect_ratio(landmarks, eye_indices, w, h):
    pts = np.array([[landmarks[i].x * w, landmarks[i].y * h] for i in eye_indices])
    return (euclidean(pts[1], pts[5]) + euclidean(pts[2], pts[4])) / (2.0 * euclidean(pts[0], pts[3]) + 1e-6)

# Индексы для глаз (FaceMesh)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # --- Pose: руки, нормализация и хлопки ---
    res_pose = pose.process(img_rgb)
    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark

        # Вытаскиваем точки и проверяем видимость
        def get_xy_vis(idx):
            p = lm[idx]
            return np.array([p.x * w, p.y * h]), p.visibility

        left_shoulder, v_ls = get_xy_vis(mp_pose.PoseLandmark.LEFT_SHOULDER.value)
        right_shoulder, v_rs = get_xy_vis(mp_pose.PoseLandmark.RIGHT_SHOULDER.value)
        left_wrist, v_lw = get_xy_vis(mp_pose.PoseLandmark.LEFT_WRIST.value)
        right_wrist, v_rw = get_xy_vis(mp_pose.PoseLandmark.RIGHT_WRIST.value)

        vis_ok = (v_ls > 0.5 and v_rs > 0.5 and v_lw > 0.5 and v_rw > 0.5)

        left_up = vis_ok and (left_wrist[1] < left_shoulder[1])
        right_up = vis_ok and (right_wrist[1] < right_shoulder[1])

        status = []
        if left_up: status.append("Left hand up")
        if right_up: status.append("Right hand up")
        if not status: status.append("....")

        # Хлопки: нормализованная дистанция по ширине кадра
        if vis_ok:
            dist = euclidean(left_wrist, right_wrist)
            dist_norm = dist / (w + 1e-6)

            # скорость сближения: положительная, если руки идут навстречу
            if prev_dist_norm is not None:
                approach = max(0.0, prev_dist_norm - dist_norm)  # только сближение
                approach_buffer.append(approach)
            prev_dist_norm = dist_norm

            # Гистерезис + требование минимальной скорости сближения
            if dist_norm < enter_thresh and (np.mean(approach_buffer) > min_approach):
                if not clap_state:
                    clap_count += 1
                    clap_state = True
            elif dist_norm > exit_thresh:
                clap_state = False

            cv2.putText(frame, f"Claps: {clap_count}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        cv2.putText(frame, ", ".join(status), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # --- MTCNN: лицо (bbox) ---
    faces = detector.detect_faces(frame)
    bbox = None
    if faces:
        x, y, w_box, h_box = faces[0]['box']
        # корректируем возможные отрицательные координаты
        x = max(0, x); y = max(0, y)
        w_box = max(0, min(w_box, w - x))
        h_box = max(0, min(h_box, h - y))
        if w_box > 0 and h_box > 0:
            bbox = (x, y, w_box, h_box)
            # для визуализации
            cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 255), 2)

    # --- FaceMesh: считаем EAR по всему кадру, но принимаем только если глаза внутри bbox MTCNN ---
    res_face = face_mesh.process(img_rgb)
    if res_face.multi_face_landmarks and bbox is not None:
        landmarks = res_face.multi_face_landmarks[0].landmark

        left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, w, h)
        right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, w, h)
        ear = (left_ear + right_ear) / 2.0

        # проверим, что глаза попадают в bbox MTCNN
        def lm_in_bbox(idx):
            px = landmarks[idx].x * w
            py = landmarks[idx].y * h
            x, y, bw, bh = bbox
            return (x <= px <= x + bw) and (y <= py <= y + bh)

        eyes_ok = lm_in_bbox(LEFT_EYE[0]) and lm_in_bbox(RIGHT_EYE[0])

        if eyes_ok:
            ear_buffer.append(ear)
            ear_avg = np.mean(ear_buffer)
            eye_status = "Eyes closed" if ear_avg < ear_thresh else "Eyes open"
            cv2.putText(frame, eye_status, (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Detection MTCNN+FaceMesh+Pose", frame)
    key = cv2.waitKey(1) & 0xFF
    if key in [27, ord('q')]:
        print("Остановка работы по клавише.")
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step
Остановка работы по клавише.


In [17]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

clap_count = 0
clap_state = False
enter_thresh = 0.15   # руки близко (<15% ширины кадра)
exit_thresh = 0.20    # руки разошлись (>20% ширины кадра)

def euclidean(a, b):
    return np.linalg.norm(a - b)

while True:
    ret, frame = cap.read()
    if not ret: break
    h, w = frame.shape[:2]
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res_pose = pose.process(img_rgb)

    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark

        # рисуем все точки Pose
        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # координаты запястий
        lw = np.array([lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w,
                       lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h])
        rw = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * w,
                       lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * h])

        # большие кружки на запястьях
        cv2.circle(frame, tuple(lw.astype(int)), 12, (0,255,0), -1)
        cv2.circle(frame, tuple(rw.astype(int)), 12, (0,0,255), -1)

        # линия между запястьями
        cv2.line(frame, tuple(lw.astype(int)), tuple(rw.astype(int)), (255,255,0), 2)

        # дистанция
        dist = euclidean(lw, rw)
        dist_norm = dist / w
        cv2.putText(frame, f"Dist_norm: {dist_norm:.3f}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (200,200,200), 2)

        # логика хлопков
        if dist_norm < enter_thresh and not clap_state:
            clap_count += 1
            clap_state = True
        elif dist_norm > exit_thresh:
            clap_state = False

        cv2.putText(frame, f"Claps: {clap_count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)

    cv2.imshow("Clap Debug", frame)
    if cv2.waitKey(1) & 0xFF in [27, ord('q')]:
        break

cap.release()
cv2.destroyAllWindows()


осанка


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

def euclidean(a, b):
    return np.linalg.norm(a - b)

def angle_between(p1, p2):
    # угол линии относительно вертикали
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = np.degrees(np.arctan2(dx, dy))  # угол наклона
    return angle

while True:
    ret, frame = cap.read()
    if not ret: break
    h, w = frame.shape[:2]
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res_pose = pose.process(img_rgb)

    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark

        # координаты плеч и бёдер
        left_shoulder = np.array([lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                                  lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h])
        right_shoulder = np.array([lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * w,
                                   lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * h])
        left_hip = np.array([lm[mp_pose.PoseLandmark.LEFT_HIP.value].x * w,
                             lm[mp_pose.PoseLandmark.LEFT_HIP.value].y * h])
        right_hip = np.array([lm[mp_pose.PoseLandmark.RIGHT_HIP.value].x * w,
                              lm[mp_pose.PoseLandmark.RIGHT_HIP.value].y * h])

        # середины плеч и бёдер
        mid_shoulder = (left_shoulder + right_shoulder) / 2
        mid_hip = (left_hip + right_hip) / 2

        # угол линии "плечи-бёдра" относительно вертикали
        angle = angle_between(mid_shoulder, mid_hip)

        # проверка осанки
        if abs(angle) < 6:  # допуск ±15°
            status = "Posture: Straight"
            color = (0, 255, 0)
        else:
            status = "Posture: Slouching"
            color = (0, 0, 255)

        cv2.putText(frame, f"{status} (angle={angle:.1f})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # рисуем линии
        cv2.circle(frame, tuple(mid_shoulder.astype(int)), 8, (255, 255, 0), -1)
        cv2.circle(frame, tuple(mid_hip.astype(int)), 8, (255, 255, 0), -1)
        cv2.line(frame, tuple(mid_shoulder.astype(int)), tuple(mid_hip.astype(int)), (255, 255, 0), 2)

        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Posture Detection", frame)
    if cv2.waitKey(1) & 0xFF in [27, ord('q')]:
        break

cap.release()
cv2.destroyAllWindows()
